In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
#!pip install pandas langchain langchain-community sentence-transformers faiss-cpu "transformers[agents]"
#!pip install "transformers[agents]"=='v4.44.0'

In [3]:
# Import necessary modules
import pandas as pd
import datasets
from transformers import AutoTokenizer
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from tqdm import tqdm
from transformers.agents import Tool,HfEngine, ReactJsonAgent
from huggingface_hub import InferenceClient
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

ModuleNotFoundError: No module named 'datasets'

In [5]:
# Load the knowledge base
knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

NameError: name 'datasets' is not defined

In [6]:
knowledge_base

NameError: name 'knowledge_base' is not defined

In [6]:
# Convert dataset to Document objects
source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

logger.info(f"Loaded {len(source_docs)} documents from the knowledge base")

#source_docs

INFO:__main__:Loaded 2647 documents from the knowledge base


In [7]:
# Initialize the text splitter
tokenizer = AutoTokenizer.from_pretrained("thenlper/gte-small")
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer,
    chunk_size=200,
    chunk_overlap=20,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

In [8]:
# Split documents and remove duplicates
logger.info("Splitting documents...")
docs_processed = []
unique_texts = {}
for doc in tqdm(source_docs):
    new_docs = text_splitter.split_documents([doc])
    for new_doc in new_docs:
        if new_doc.page_content not in unique_texts:
            unique_texts[new_doc.page_content] = True
            docs_processed.append(new_doc)

logger.info(f"Processed {len(docs_processed)} unique document chunks")

INFO:__main__:Splitting documents...
100%|██████████| 2647/2647 [00:36<00:00, 73.32it/s] 
INFO:__main__:Processed 43181 unique document chunks


In [61]:
# Initialize the embedding model
logger.info("Initializing embedding model...")
embedding_model = HuggingFaceEmbeddings(model_name="thenlper/gte-small")

# Create the vector database
logger.info("Creating vector database...")
vectordb = FAISS.from_documents(
    documents=docs_processed,
    embedding=embedding_model,
    distance_strategy=DistanceStrategy.COSINE,
)

logger.info("Vector database created successfully")

INFO:__main__:Initializing embedding model...
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: thenlper/gte-small
INFO:__main__:Creating vector database...
INFO:__main__:Vector database created successfully


In [62]:
save_path = "./faiss_index"
vectordb.save_local(save_path)

In [9]:
# load Vector DB
faiss_path = "./faiss_index"
embedding_model = HuggingFaceEmbeddings(model_name="thenlper/gte-small")

vectordb= FAISS.load_local(faiss_path,embeddings=embedding_model,allow_dangerous_deserialization=True)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: thenlper/gte-small
INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.


## Building the Retriever Tools

In [10]:
class RetrieverTool(Tool):
    name = "retriever"
    description = "Using semantic similarity, retrieves some documents from the knowledge base that have the closest embeddings to the input query."
    inputs = {
        "query": {
            "type": "text",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "text"

    def __init__(self, vectordb, **kwargs):
        super().__init__(**kwargs)
        self.vectordb = vectordb

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.vectordb.similarity_search(
            query,
            k=7,
        )

        return "\nRetrieved documents:\n" + "".join(
            [f"===== Document {str(i)} =====\n" + doc.page_content for i, doc in enumerate(docs)]
        )

# Create an instance of the RetrieverTool
retriever_tool = RetrieverTool(vectordb)

In [11]:
result = retriever_tool.forward("How can I join the community?")
print("result : ",result)

result :  
Retrieved documents:
===== Document 0 =====
.fast.ai/). Join the fast.ai [Discord](https://discord.com/invite/YKrxeNn) and [forums](https://forums.fast.ai/). It is a guarantee that you will learn by being part of their community!===== Document 1 =====
## Community===== Document 2 =====
<a href="https://huggingface.co/new-space?template=Panel-Org/panel-template">
<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/hub/spaces-panel.png" style="width:70%"> </a>


## 🌐 Join Our Community
The Panel community is vibrant and supportive, with experienced developers and data scientists eager to help and share their knowledge. Join us and connect with us:===== Document 3 =====
<iframe width="560" height="315" src="https://www.youtube.com/embed/XvSGPZFEjDY" title="YouTube video player"
frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope;
picture-in-picture" allowfullscreen></iframe>

<Tip>

To share a model wit

In [13]:
# Initialize the language model engine
"""
llm_engine = HfEngine("meta-llama/Meta-Llama-3-8B-Instruct")
"""

'\nllm_engine = HfEngine("meta-llama/Meta-Llama-3-8B-Instruct")\n'

In [82]:
# ### IF YOU WANT TO USE OPENAI
"""
import os
from openai import OpenAI

from typing import List, Dict
from transformers.agents.llm_engine import MessageRole, get_clean_message_list
from huggingface_hub import InferenceClient

openai_role_conversions = {
    MessageRole.TOOL_RESPONSE: MessageRole.USER,
}


class OpenAIEngine:
    def __init__(self, model_name="gpt-4o"):
        self.model_name = model_name
        self.client = OpenAI(
            api_key="YOUR-API-KEY"#os.getenv("OPENAI_API_KEY"),
        )

    def __call__(self, messages, stop_sequences=[]):
        messages = get_clean_message_list(messages, role_conversions=openai_role_conversions)

        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=messages,
            stop=stop_sequences,
            temperature=0.5,
        )
        return response.choices[0].message.content

"""

'\nimport os\nfrom openai import OpenAI\n\nfrom typing import List, Dict\nfrom transformers.agents.llm_engine import MessageRole, get_clean_message_list\nfrom huggingface_hub import InferenceClient\n\nopenai_role_conversions = {\n    MessageRole.TOOL_RESPONSE: MessageRole.USER,\n}\n\n\nclass OpenAIEngine:\n    def __init__(self, model_name="gpt-4o"):\n        self.model_name = model_name\n        self.client = OpenAI(\n            api_key="YOUR-API-KEY"#os.getenv("OPENAI_API_KEY"),\n        )\n\n    def __call__(self, messages, stop_sequences=[]):\n        messages = get_clean_message_list(messages, role_conversions=openai_role_conversions)\n\n        response = self.client.chat.completions.create(\n            model=self.model_name,\n            messages=messages,\n            stop=stop_sequences,\n            temperature=0.5,\n        )\n        return response.choices[0].message.content\n\n'

In [54]:
# process with LLM Studio
from openai import OpenAI

class LLMStudioEngine:
     def __init__(self) -> None:
         pass
     
     def __call__(self,messages, stop_sequences=[]):
  
        client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
        messages=[
                {"role": "system", "content": "Always answer in rhymes."},
                {"role": "user", "content": messages}
                ]
        completion = client.chat.completions.create(
            model="TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF",
            messages=messages,
            stop=stop_sequences,
            temperature=0.7,
            )
        
        return completion.choices[0].message.content


In [55]:
### Test LLM
llm_engine = LLMStudioEngine()


In [56]:
# Create the agent
agent = ReactJsonAgent(tools=[retriever_tool], llm_engine=llm_engine, max_iterations=4, verbose=2)

In [57]:
# Function to run the agent
def run_agentic_rag(question: str) -> str:
    enhanced_question = f"""Using the information contained in your knowledge base, which you can access with the 'retriever' tool,
give a comprehensive answer to the question below.
Respond only to the question asked, response should be concise and relevant to the question.
If you cannot find information, do not give up and try calling your retriever again with different arguments!
Make sure to have covered the question completely by calling the retriever tool several times with semantically different queries.
Your queries should not be questions but affirmative form sentences: e.g. rather than "How do I load a model from the Hub in bf16?", query should be "load a model from the Hub bf16 weights".

Question:
{question}"""

    return agent.run(enhanced_question)


In [58]:
# Example usage
question = "How can I push a model to the Hub?"
answer = run_agentic_rag(question)
print(f"Question: {question}")
print(f"Answer: {answer}")

======== New task ========
Using the information contained in your knowledge base, which you can access with the 'retriever' tool,
give a comprehensive answer to the question below.
Respond only to the question asked, response should be concise and relevant to the question.
If you cannot find information, do not give up and try calling your retriever again with different arguments!
Make sure to have covered the question completely by calling the retriever tool several times with semantically different queries.
Your queries should not be questions but affirmative form sentences: e.g. rather than "How do I load a model from the Hub in bf16?", query should be "load a model from the Hub bf16 weights".

Question:
How can I push a model to the Hub?
System prompt is as follows:
You are an expert assistant who can solve any task using JSON tool calls. You will be given a task to solve as best you can.
To do so, you have been given access to the following tools: 'retriever', 'final_answer'
The 

Question: How can I push a model to the Hub?
Answer: Error in generating final llm output: Error code: 400 - {'error': "Invalid 'content': 'content' objects must have a 'type' field that is either 'text' or 'image_url' Got 'object'."}.


In [59]:
# Example usage
question = "How to install Huggingface lib?"
answer = run_agentic_rag(question)
print(f"Question: {question}")
print(f"Answer: {answer}")

======== New task ========
Using the information contained in your knowledge base, which you can access with the 'retriever' tool,
give a comprehensive answer to the question below.
Respond only to the question asked, response should be concise and relevant to the question.
If you cannot find information, do not give up and try calling your retriever again with different arguments!
Make sure to have covered the question completely by calling the retriever tool several times with semantically different queries.
Your queries should not be questions but affirmative form sentences: e.g. rather than "How do I load a model from the Hub in bf16?", query should be "load a model from the Hub bf16 weights".

Question:
How to install Huggingface lib?
System prompt is as follows:
You are an expert assistant who can solve any task using JSON tool calls. You will be given a task to solve as best you can.
To do so, you have been given access to the following tools: 'retriever', 'final_answer'
The way

Question: How to install Huggingface lib?
Answer: Error in generating final llm output: Error code: 400 - {'error': "Invalid 'content': 'content' objects must have a 'type' field that is either 'text' or 'image_url' Got 'object'."}.
